# Apriori Algorithm implementation in Python

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import time

## Data Source:
http://fimi.ua.ac.be/data/
    
Data set used here: 

     Given by Ferenc Bodon, contains (anonymized) click-stream data of a hungarian on-line news portal.

In [2]:
filename = 'kosarak.dat.txt'

## Loading dataset:
By default only, 10K rows read

In [3]:
def load_data(filename, no_lines = 10000):
    """
    Input is a space separated text filename
    Function outputs list of sets
    """
    l = []
    count = 0
    with open(filename) as f:
        for line in f:
            l.append(set([int(ele) for ele in line.replace('\n','').split()]))
            count+=1
            if count > no_lines:
                print('Data read into list of sets')
                break
    
    return l


basket_data = load_data(filename)

Data read into list of sets


In [4]:
# basket_data[0]

## More about Algorithm:

Aim: Find frequent Itemsets (Frequent subsets with one or more items)

Pseudo code:

1) Generate list of items with frequency above the minimum threshold (# times a basket contains the item) 

2) Based on this subset, look for subsets of basket with size = 2, whose frequency crosses the threshold 

3) Continue this until you get a subset that don't fit the threshold

<img src="apriori.png">

One important thing to note is the ... property. Which states that ...

In [5]:
def create_prod_list(data):
    """
    Initial pass through the data, to check the number of unique items
    Returns a list of items stored as frozensets
    Frozen set is used to avoid mutability and also as these groups are revisited again.
    """
    prod_list = []
    prod_set = set()
    for basket in data:
        for prod in basket:
            if prod not in prod_set:
                prod_set.add(prod)
                prod_list.append([prod])
    print('Product list created!')
    return list(map(frozenset,prod_list))

# c1 = create_prod_list(basket_data)

In [6]:
def create_lk(dataset, ck, threshold=0.01*10000):
    """
    Checks for every element in ck, if a subset of basket, keep a count.
    Passes on the sets which have count over threshold
    """
    freq_count = {}
    for basket in dataset:
        for pair in ck:
            if pair.issubset(basket):
                if pair in freq_count:
                    freq_count[pair]+=1
                else:
                    freq_count[pair]=1
    
    data_size = len(dataset)
    sort_count = sorted(freq_count.items(), key=lambda x:x[1], reverse = True)
    lk = []
    support_dict = {}
    for ele in sort_count:
        if ele[1] >= threshold:
            lk.append(ele[0])
            support_dict[ele[0]] = ele[1]/data_size
#     print("Created L set")
    lk.sort()
    return lk, dict(support_dict)
    
# l1, supp_dict = create_lk(basket_data, c1)

In [7]:
def create_ck(lk,k):
    """
    two sets from L that differ in just one element will be joined together.
    """
    retList = []
    lenlk = len(lk)
    for i in range(lenlk):
        for j in range(i+1, lenlk): 
            tmp_l1 = list(lk[i])[:k-2]
            tmp_l2 = list(lk[j])[:k-2]
            if tmp_l1==tmp_l2: #if first k-1 elements are equal
                retList.append(lk[i] | lk[j]) #set union
#     print("Created C set")
    return retList

In [10]:
def apriori(data, threshold=0.01*10000, break_early = False):
    """
    Main function for the algortithm
    Break_early is used to stop the algorithm until the sets of 2
    """
    print("Start Apriori algorithm")
    c1 = create_prod_list(data)
    l1, support_data = create_lk(data, c1, threshold)
    L = [l1]
    print("Value for K:"+str(1))
    print("Number of freqent itemsets: " + str(len(l1)))
    k = 2
    while (len(L[k-2]) > 0):
        print("Value for K:"+str(k))
        t = time.time()
        ck = create_ck(L[k-2], k)
        lk, supK = create_lk(data, ck, threshold)#scan DB to get Lk
        support_data.update(supK)
        L.append(lk)
        k += 1
        print("Time taken in secs: "+str((time.time()-t)))
        print("Number of freqent itemsets: " + str(len(lk)))
        if break_early and k >3:
            break
    return L, support_data

In [11]:
sets, support_info = apriori(basket_data, threshold=0.01*len(basket_data), break_early = False)

Start Apriori algorithm
Product list created!
Value for K:1
Number of freqent itemsets: 54
Value for K:2
Time taken in secs: 1.324998378753662
Number of freqent itemsets: 137
Value for K:3
Time taken in secs: 0.4530012607574463
Number of freqent itemsets: 127
Value for K:4
Time taken in secs: 0.10899925231933594
Number of freqent itemsets: 37
Value for K:5
Time taken in secs: 0.008998632431030273
Number of freqent itemsets: 5
Value for K:6
Time taken in secs: 0.0010008811950683594
Number of freqent itemsets: 0


## Association Rules

Rules are defined using confidence as a metric.

Confidence:

\begin{equation*}
I -> j = \frac{support({I} + {j})}{support({I})} - \frac{support({j})}{countOfBaskets}
\end{equation*}

The frequent itemsets and the support values are used to write rules.

In [12]:
def calculate_conf(itemset, H, support_data, rl, conf=0.7, verbose=False):
    """Calulates the confidence for each pair"""
    prunedH = []
    for conseq in H:
        tmp_conf = support_data[itemset]/support_data[itemset-conseq] 
        if tmp_conf >= conf: 
            if verbose:
                print (itemset-conseq,'-->',conseq,'conf:',tmp_conf)
            rl.append((itemset-conseq, conseq, tmp_conf))
            prunedH.append(conseq)
    return prunedH

In [13]:
def rulesFromConseq(itemset, H, support_data, rl, conf=0.7, verbose=False):
    m = len(H[0])
    if (len(itemset) > (m + 1)):    
        Hmp1 = create_ck(H, m+1)     
        Hmp1 = calculate_conf(itemset, Hmp1, support_data, rl, conf, verbose)
        if (len(Hmp1) > 1):    
            rulesFromConseq(itemset, Hmp1, support_data, rl, conf, verbose)

In [14]:
def generateRules(frequent_itemsets, support_data, conf=0.7, verbose = False): 
    rule_list = []
    for i in range(1, len(frequent_itemsets)):
        for itemset in frequent_itemsets[i]:
            H1 = [frozenset([item]) for item in itemset]
            if (i > 1):
                rulesFromConseq(itemset, H1, support_data, rule_list, conf, verbose)
            else:
                calculate_conf(itemset, H1, support_data, rule_list, conf, verbose)
    return rule_list

In [16]:
apriori_rule_list = generateRules(sets, support_info, conf=0.7, verbose = False)

## Checking for small data

In [17]:
tmp_data = [set(ele) for ele in [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]]
tmp_sets, tmp_support_info = apriori(tmp_data, threshold=2)

tmp_rule_list = generateRules(tmp_sets, tmp_support_info, conf=0.7)


Start Apriori algorithm
Product list created!
Value for K:1
Number of freqent itemsets: 4
Value for K:2
Time taken in secs: 0.00098419189453125
Number of freqent itemsets: 4
Value for K:3
Time taken in secs: 0.0
Number of freqent itemsets: 1
Value for K:4
Time taken in secs: 0.0
Number of freqent itemsets: 0
